In [1]:
!pip install transformers accelerate torch torchvision pillow tqdm pandas scikit-learn evaluate bert-score rouge-score rapidfuzz sentence-transformers inflect

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   -- ------------------------------------- 1.6/25.7 MB 8.5 MB/s eta 0:00:03
   ----- ---------------------------------- 3.4/25.7 MB 8.6 MB/s eta 0:00:03
   -------- ------------------------------- 5.5/25.7 MB 9.1 MB/s eta 0:00:03
   ----------- ---------------------------- 7.3/25.7 MB 9.3 MB/s eta 0:00:02
  

In [3]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
from PIL import Image
import torch
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import time

In [3]:
train_df = pd.read_csv('/kaggle/input/datasetnewsplit/train.csv')
val_df = pd.read_csv('/kaggle/input/datasetnewsplit/val.csv')
test_df = pd.read_csv('/kaggle/input/datasetnewsplit/test.csv')
print(f"Loaded training dataset with {len(train_df)} entries")
print(f"Loaded validation dataset with {len(val_df)} entries")
print(f"Loaded testing dataset with {len(test_df)} entries")

Loaded training dataset with 80001 entries
Loaded validation dataset with 20001 entries
Loaded testing dataset with 20001 entries


In [4]:
val_df["full_image_path"] = val_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))
val_df["full_image_path"] = val_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
train_df["full_image_path"] = train_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))
train_df["full_image_path"] = train_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
test_df["full_image_path"] = test_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
test_df["full_image_path"] = test_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "llava-hf/bakLlava-v1-hf"
processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16)
model.eval().to(device)

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/934M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): Q

In [6]:
# VERIFY MODEL TYPE IMMEDIATELY AFTER LOADING
print(f"Type of 'model' after loading: {type(model)}")
assert isinstance(model, LlavaForConditionalGeneration), "Model is not a LlavaForConditionalGeneration instance!"

Type of 'model' after loading: <class 'transformers.models.llava.modeling_llava.LlavaForConditionalGeneration'>


In [7]:
import re
import inflect
inflect_engine = inflect.engine()

def normalize_answer(ans):
    ans = ans.strip().lower()
    ans = re.sub(r'[^\w\s]', '', ans)  # remove punctuation
    if ans.isdigit():
        ans = inflect_engine.number_to_words(ans)
    words = ans.split()
    return words[0] if words else ""  # return first word or empty string

In [8]:
correct = 0
total = 0
all_preds = []
all_labels = []
start_time = time.time()
TIME_LIMIT= 42000

In [9]:
import os
os.makedirs("/kaggle/working/resultCSV", exist_ok=True)
pred_path = '/kaggle/working/resultCSV/predictions.csv'

In [10]:
results = []
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    elapsed_time = time.time()
    if elapsed_time - start_time > TIME_LIMIT:
            print(f"\nTime limit of {TIME_LIMIT} seconds exceeded. Aborting loop at index {idx}.")
            print(f'Number of successfully run images - {idx}')
            break
    image_path = Path(row['full_image_path'])
    if not image_path.exists():
        print(f"Image {image_path} does not exist.")
        continue

    try:
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Failed to load image at {image_path}: {e}")
        continue

    question = row['question']
    gt_answer = normalize_answer(row['answer'])

    # Prompt construction
    prompt = f"<image>\nBased on the image, answer the following question with a single word. Question: {question} Answer:"
    
    try:
        inputs = processor(text=prompt, images=image, return_tensors="pt", padding=True)
        # Move to device and handle float16
        inputs = {k: v.to(device) for k, v in inputs.items()}
        for k in inputs:
            if k not in ["input_ids", "attention_mask"]:
                inputs[k] = inputs[k].to(torch.float16)

        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=10)

        pred_full = processor.batch_decode(output, skip_special_tokens=True)[0]
        # Extract prediction after colon, then clean it
        if ":" in pred_full:
            pred_text = pred_full.split(":")[-1].strip()
        else:
            pred_text = pred_full.strip()

        pred_words = pred_text.split()
        pred_answer = normalize_answer(pred_words[0]) if pred_words else ""

    except Exception as e:
        print(f"Inference failed at idx {idx} with error: {e}")
        continue

    all_preds.append(pred_answer)
    all_labels.append(gt_answer)

    if pred_answer == gt_answer:
        correct += 1
    total += 1

    # print(f"Q: {question} | GT: {gt_answer} | Pred: {pred_answer}")
    # cnt +=1 
    # if(cnt == 5):
    #     break
    results.append({
        "img_path": image_path,
        "question": question,
        "true_answer": str(gt_answer).lower(),
        "predicted_answer": str(pred_answer).lower()
    })
        # print(f"\nIdx: {idx}")
        # print(f"Original Question: {question}")
        # print(f"Full Prediction: '{pred_full}'")
        # print(f"Processed Prediction (Single Word Attempt): '{pred}'")
        # print(f"Answer ground truth: '{gt_answer}'")
        #print("count: ", count)
        # if (count == 100): # Limiting to 100 for the example
        #     print("\nReached count limit (100). Breaking loop.")
        #     break
    if (idx + 1) % 1000 == 0 or (idx + 1) == len(test_df):
    # Convert to DataFrame
        chunk_df = pd.DataFrame(results)

        # Save predictions
        if not os.path.exists(pred_path):
            chunk_df.to_csv(pred_path, mode='w', index=False, header=True)
        else:
            chunk_df.to_csv(pred_path, mode='a', index=False, header=False)

        print(f"Checkpoint saved at index {idx + 1}")
        # Reset for next chunk
        results = []

# Final accuracy print
# print(f"\nFinal Accuracy: {correct}/{total} = {100 * correct / max(1, total):.2f}%")

  5%|▍         | 1000/20001 [10:40<3:27:50,  1.52it/s]

Checkpoint saved at index 1000


 10%|▉         | 2000/20001 [21:26<3:21:30,  1.49it/s]

Checkpoint saved at index 2000


 15%|█▍        | 3000/20001 [32:15<2:55:08,  1.62it/s]

...

Checkpoint saved at index 19000


100%|█████████▉| 20000/20001 [3:36:28<00:00,  1.49it/s]

Checkpoint saved at index 20000


100%|██████████| 20001/20001 [3:36:29<00:00,  1.54it/s]

Checkpoint saved at index 20001


In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
from bert_score import BERTScorer
from rouge_score import rouge_scorer
from rapidfuzz.distance import Levenshtein
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

# Normalize case


# Normalize case
# Normalize case
preds = pd.read_csv('/kaggle/working/resultCSV/predictions.csv')
preds = preds.dropna()
# preds = preds.iloc[1:].reset_index(drop=True)
all_preds = preds['predicted_answer'].tolist()
all_labels = preds['true_answer'].tolist()

In [12]:


# Normalize case
preds_l = [p.lower() for p in all_preds]
refs_l = [r.lower() for r in all_labels]

# Compute exact-match binary metrics
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1] * len(all_labels)

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)

# Print metrics
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}")
print(f"Exact-match Recall:    {rec:.3f}")
print(f"Exact-match F1:        {f1:.3f}\n")

# Save predictions and ground truths to CSV
# pred_ref_df = pd.DataFrame({
#     "Prediction": predictions,
#     "Ground_Truth": refs
# })
# df.to_csv('prediction_output.csv', index=False)

Exact-match Accuracy: 0.494
Exact-match Precision: 1.000
Exact-match Recall:    0.494
Exact-match F1:        0.661



In [9]:
from bert_score import score

P, R, F1 = score(preds_l, refs_l, lang="en", verbose=True, rescale_with_baseline = True)
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/31 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/313 [00:00<?, ?it/s]

done in 19.97 seconds, 1001.70 sentences/sec
BERTScore - Precision: 0.9145, Recall: 0.8879, F1: 0.8995


In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rougeL_scores = []

for pred, ref in zip(preds_l, refs_l):
    scores = scorer.score(ref, pred)
    rouge1_scores.append(scores["rouge1"].fmeasure)
    rougeL_scores.append(scores["rougeL"].fmeasure)

print(f"ROUGE Scores:")
print(f"ROUGE-1 F1: {np.mean(rouge1_scores):.4f}")
print(f"ROUGE-L F1: {np.mean(rougeL_scores):.4f}")

ROUGE Scores:
ROUGE-1 F1: 0.5038
ROUGE-L F1: 0.5038


In [12]:
lev_dists = [Levenshtein.distance(p, r) for p, r in zip(preds_l, refs_l)]
avg_lev = np.mean(lev_dists)
print(f"Levenshtein Distance:")
print(f"Average Edit Distance: {avg_lev:.2f}")

Levenshtein Distance:
Average Edit Distance: 2.64


In [14]:
# 5. Sentence Embedding Cosine Similarity (memory-efficient pairwise)
model = SentenceTransformer('all-MiniLM-L6-v2')
pred_embeds = model.encode(preds_l, convert_to_tensor=True, device='cuda')
ref_embeds = model.encode(refs_l, convert_to_tensor=True, device='cuda')

cos_sims = [
    util.cos_sim(pred_embeds[i], ref_embeds[i]).item()
    for i in range(len(preds_l))
]

print(f"Sentence-BERT Cosine Similarity:")
print(f"Average Cosine Similarity: {np.mean(cos_sims):.4f}")


Sentence-BERT Cosine Similarity:
Average Cosine Similarity: 0.7514
